In [23]:
import pandas as pd
# import spacy
import torch


df = pd.read_csv("contrib_from_csv.csv", sep = ",", dtype= str)

# sample d'un très petit échantillon pour cet ordi
df = df.sample(1000)

In [24]:
# pour nettoyer les divers breakline marks et les mots notés illisibles

join_breaklines = lambda x : x.replace("\x0b"," ") 
join_breaklinesn = lambda x : x.replace("\n"," ") 
kill_texte_illisible = lambda x : x.replace("texte illisible", "")
kill_illisible = lambda x : x.replace("illisible", "")
kill_illisible_plural = lambda x : x.replace("illisibles", "")

In [25]:
df["clean"] = df["Contribution"].apply(join_breaklines)
df["clean"] = df["clean"].apply(join_breaklinesn)
df["clean"] = df["clean"].apply(kill_texte_illisible)
df["clean"] = df["clean"].apply(kill_illisible)
df["clean"] = df["clean"].apply(kill_illisible_plural)

In [26]:
pd.set_option('display.max_colwidth', None)
df.head()

,Catégorie,Date de réception,Code postal,Code INSEE,Numéro d'ordre arbitraire,Type Graphie TT,Numéro de page,Numéro séquentiel,Contribution,joined_id,clean
38229,CC,190225,85540,85206,07078,MD,11,5,"Service des urgences: Établir des priorités pour les prises en charges: directions nationales ex: refuser de soigner dans [mot illisible] ou va notre argent? Qui coûte cher? mes parlementaires eu limiter le nombre nos hauts fonctionnaires nos retraités de l'armée nos anciens présidents de la République: ex: nos anciens Présidents à la retraite, même si depuis octobre 2016, institue la dégressivité des moyens dévolus à ,nos ex présidents Valéry Giscard d'Etaing: dépense de l'Etat 3,9 millions £/an Nicolas Larkazy: dépense de l'Etat 3,3 millions £/an Jacques Chirac: dépense de l'Etat 2,4 millions £/an 9,6 millions £/an Que se passera t-il pour E. MACRON? il aura 44 ans dans 5 ans Le contribuable doit payer pendant de nombreux années!!!...",CC_85540_190225_85206_MD_07078,"Service des urgences: Établir des priorités pour les prises en charges: directions nationales ex: refuser de soigner dans [mot ] ou va notre argent? Qui coûte cher? mes parlementaires eu limiter le nombre nos hauts fonctionnaires nos retraités de l'armée nos anciens présidents de la République: ex: nos anciens Présidents à la retraite, même si depuis octobre 2016, institue la dégressivité des moyens dévolus à ,nos ex présidents Valéry Giscard d'Etaing: dépense de l'Etat 3,9 millions £/an Nicolas Larkazy: dépense de l'Etat 3,3 millions £/an Jacques Chirac: dépense de l'Etat 2,4 millions £/an 9,6 millions £/an Que se passera t-il pour E. MACRON? il aura 44 ans dans 5 ans Le contribuable doit payer pendant de nombreux années!!!..."
89485,CC,190322,78850,78615,19644,MD,5,3,"Le 11/01/2019\nPuisqu'une famille peut très bien vivre alors que le mariage ne touche que le SMIC. Pourquoi nos Parlementaires ne seraient-ils pas rétribués ainsi ce qui ferait. Je pense de très importantes économies au lieu d'aller piocher dans la poche des retraites. D'autre part que l'on arrête un peu de rémunérer dans les guerres internationales. Depuis 1989, la France est en Guerre que l'un arrête un peu ces rémunérés, en attendant c'est le peuple qui paie.",CC_78850_190322_78615_MD_19644,"Le 11/01/2019 Puisqu'une famille peut très bien vivre alors que le mariage ne touche que le SMIC. Pourquoi nos Parlementaires ne seraient-ils pas rétribués ainsi ce qui ferait. Je pense de très importantes économies au lieu d'aller piocher dans la poche des retraites. D'autre part que l'on arrête un peu de rémunérer dans les guerres internationales. Depuis 1989, la France est en Guerre que l'un arrête un peu ces rémunérés, en attendant c'est le peuple qui paie."
107664,CC,190228,07500,07102,15164,MD,24,4,"Cahier de doléances :\nMadame Oebanne Claude\nJ'apporte ma « pierre à l'édifice » que devrait être ce débat.\nIl ne peut en être autrement ayant toujours voté socialiste, milité et exercé avec passion mon rôle de professeur des collèges.\nJe suis horrifiée par la violence, la haine qui ravage notre pays !\nQue veulent ces « gilets jaunes ?» Le savent-ils ? Ont-ils conscience qu'ils sont des jouets entre les\nmains d'extrémistes ?\nPlus de justice sociale devant l'impôt : tout le monde devrait payer celui sur le revenu, même une somme symbolique pour les plus démunis.\nLa CSG devrait être comprise dans l'IR pour plus de progressivité.\nPlus de justice sociale en « mettant le paquet » sur l'école. Bravo pour le dédoublement de certains\nCP ; mais je regrette beaucoup que Mr. Blanquer ait supprimé les rythmes scolaires du quinquennat\nprécédent. Ils prenaient en compte l'intérêt de l'enfant : apprentissage le matin et surtout des\nactivités culturelles, sportives et artistiques en 2'^™ partie de journée. Les communes n'ont pas joué le jeu, trop souvent par idéologie!!!Où est l'égalité entre celui qui peut offrir à ses enfants des cours au conservatoire, la licence de tennis ...et celui qui gagne le SMIC ????? san

In [27]:
documents = df.clean.tolist()
documents[0]

"Service des urgences:  Établir des priorités pour les prises en charges: directions nationales  ex: refuser de soigner dans [mot ] ou va notre argent? Qui coûte cher?  mes parlementaires eu limiter le nombre nos hauts fonctionnaires nos retraités de l'armée nos anciens présidents de la République: ex: nos anciens Présidents à la retraite, même si depuis octobre 2016, institue la dégressivité des moyens dévolus à ,nos ex présidents Valéry Giscard d'Etaing: dépense de l'Etat 3,9 millions £/an  Nicolas Larkazy: dépense de l'Etat 3,3 millions £/an Jacques Chirac: dépense de l'Etat 2,4 millions £/an  9,6 millions £/an Que se passera t-il pour E. MACRON? il aura 44 ans dans 5 ans  Le contribuable doit payer pendant de nombreux années!!!..."

In [28]:
# sampling 0.1/1 for counting words without crashing the kernel : 

# out_serie = random_sample.str.split().str.len()
# print("nb de mots par contribution en moyenne", out_serie.sum()/out_serie.size)

# nlp = spacy.load("fr_core_news_md")
# for_cell = lambda x : nlp(x)
# df["data"] = random_sample.apply(for_cell)
# fait aussi crasher le kernel local

In [29]:
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
from spacy.lang.en.stop_words import STOP_WORDS as en_stop

final_stopwords_list = list(fr_stop) + list(en_stop) + ["\n"]

def remove_stopwords(text):
    stopwords = final_stopwords_list
    text = [word for word in text if word not in stopwords]
    return text

In [30]:
nostopword = remove_stopwords(documents)

In [31]:
import spacy

In [32]:
def lemmatize(docs, allowed_postags = ["NOUN", "ADJ", "VERB", "ADV"]):
  '''
  Performs lemmization of input documents.
  Args:
    - docs: list of strings with input documents
    - allowed_postags: list of accepted Part of Speech (POS) types
  Output:
    - list of strings with lemmatized input
  '''
  nlp = spacy.load("fr_core_news_md", disable = ["parser", "ner"])
  nlp.max_length = 3000000 #this is possible as we're not using parser or ner.
  lemmatized_docs = []
  for doc in docs:
    doc = nlp(doc)
    tokens = []
    for token in doc:
      if token.pos_ in allowed_postags:
        tokens.append(token.lemma_)
    lemmatized_docs.append(" ".join(tokens))
  return (lemmatized_docs)

In [33]:
datas = lemmatize(nostopword)

In [34]:
from top2vec import Top2Vec

In [35]:
model = Top2Vec(datas)

2023-04-12 02:31:00,241 - top2vec - INFO - Pre-processing documents for training
2023-04-12 02:31:00,907 - top2vec - INFO - Creating joint document/word embedding
2023-04-12 02:31:13,046 - top2vec - INFO - Creating lower dimension embedding of documents
2023-04-12 02:31:19,452 - top2vec - INFO - Finding dense areas of documents
2023-04-12 02:31:19,529 - top2vec - INFO - Finding topics


In [36]:
topic_words, word_scores, topic_nums = model.get_topics()

In [37]:
for topic_word, word_score, topic_num in zip(topic_words, word_scores, topic_nums)  :
    print(topic_word)
    print(word_score)
    print(topic_num)

['effort' 'tout' 'passer' 'remettre' 'assurer' 'premier' 'vie' 'continuer'
 'gouvernement' 'meilleur' 'autre' 'retour' 'temps' 'medical' 'moyen'
 'instaurer' 'argent' 'meme' 'mme' 'zone' 'diminuer' 'point' 'fin' 'base'
 'interet' 'servir' 'systeme' 'macron' 'haut' 'mettre' 'partir' 'ecole'
 'reste' 'train' 'travail' 'moins' 'reduction' 'francais' 'soin' 'bien'
 'justice' 'augmenter' 'prendre' 'vrai' 'seulement' 'etc' 'mesure'
 'indemnite' 'campagne' 'compte']
[0.8851685  0.8754102  0.86307204 0.8616712  0.8567029  0.85337496
 0.85330933 0.84307563 0.8415454  0.83615816 0.8258957  0.82552654
 0.82361317 0.81941414 0.81720454 0.8164168  0.8134489  0.8129446
 0.8118741  0.8099179  0.8042088  0.80201554 0.80123055 0.80052793
 0.7996493  0.79934967 0.7983122  0.79332    0.7922857  0.7913057
 0.7911997  0.79107374 0.78910786 0.7883245  0.78767246 0.7864764
 0.7861397  0.785542   0.78375804 0.7826498  0.7818991  0.7808412
 0.7785379  0.7784716  0.778361   0.7776022  0.77541083 0.7750343
 0.77

On souhaite relier les topics selon les lieux géographiques. 
valence et excitation pour les aspects des thèmes mais complexe à cartographier. 
En appliquant un modèle particulièrement souple pour les petits corpus sur chaque region / departement. 
Ou bien clustering des thèmes et sentiments + labellisation pour chaque contributions texte -> puis une localisation d'ensembles géographiques grâce aux INSEE associés.
Les limites : beaucoup de clustering automatique => il faudrait faire une labellisation manuelle pour faire une évaluation des succès de ces clusterisation

In [22]:
documents, document_score, document_id = model.search_documents_by_topic(topic_num = 0, num_docs = 10)
for doc, score, doc_id in zip(documents, document_score, document_id) :
    print(f"Document  : {doc_id}, Score : {score}")
    print("___________________")
    print(doc)
    print("___________________")

Document  : 500, Score : 0.9881346821784973
___________________
j)jluyo^ rv\a(x^ 7- o grand débat national reunion ï i. transition écologique i. dimanche mars 15h 18h maison ouvrir tout venir participer réunion minute - commonoufé réunion faire partie cyde organiser initiative groupe citoyen dernier réunion avoir lieu vendredi mars tence salle 20h porter ensemble thème ouvrir débat pouvoir retrouver info presse local grand débat national
___________________
Document  : 784, Score : 0.9878599047660828
___________________
m. chef oui augmentation prix carburant pas véhicule électrique vouloir moteur hydrogène savoir faire aide achat véhicule pouvoir parler transition énergétique défiscalisation heure supplémentaire mesure maintenir temps délai quinquennat
___________________
Document  : 270, Score : 0.9860568642616272
___________________
année université début année fréquence transport commun ne pas revoir hausse croissance population inciter gens déplacer voiture éloignement société sus

In [10]:
word_scores

array([[0.86361754, 0.85665566, 0.8444204 , 0.8398157 , 0.8367531 ,
        0.82763207, 0.8225254 , 0.8090654 , 0.8054184 , 0.80409557,
        0.7968753 , 0.79481846, 0.7884388 , 0.78179294, 0.77850837,
        0.77790725, 0.7768027 , 0.77023965, 0.76532096, 0.76470566,
        0.76081926, 0.7602687 , 0.7594579 , 0.7586243 , 0.75664234,
        0.75634044, 0.75532186, 0.7540948 , 0.75344354, 0.75246894,
        0.75152016, 0.75070286, 0.7504488 , 0.7498183 , 0.7488956 ,
        0.7484929 , 0.74659777, 0.74609196, 0.7456746 , 0.7456562 ,
        0.7450966 , 0.7433404 , 0.74332035, 0.74307644, 0.7419429 ,
        0.73967916, 0.7381893 , 0.7377421 , 0.7376767 , 0.7367153 ],
       [0.8790553 , 0.8694637 , 0.852141  , 0.84321046, 0.8355927 ,
        0.8209332 , 0.8106538 , 0.8022459 , 0.7988946 , 0.77172977,
        0.77035356, 0.7644616 , 0.7543482 , 0.74540514, 0.73439115,
        0.731343  , 0.7240042 , 0.72203493, 0.7163697 , 0.7097341 ,
        0.7031176 , 0.6780646 , 0.67707556, 0.6

In [11]:
topic_nums

array([0, 1, 2])

In [39]:
random_sample = df["Contribution"].sample(frac = 0.10)
mots_liste = random_sample.tolist()
# random_sample.to_csv("chunk_df.csv", index=False)
print(lemmatize(mots_liste[0:10]))

["vraiment besoin pédagogie an infusion poudre autre concoction perlimpinpin laboratoire apprenti sorcier payer denier constater merde mettre ne comprendre doute pas grand chose finir vraiment demander avoir réellement chose compréhensible comprendre gestion finance public cours dernier année ne étonnant adduction théorème danaïde solliciter bien tonneau satisfaire infini besoin ne reste plus trou combler enfant énorme femme homme etat censé mériter légitimement respect haut niveau érudition science technicité savoir intelligence néanmoins amener milliard euro dette falloir aujourd'hui gérer encore mot gérer bien présomptueux digérer doute plus propos comment relancer partie monopoly arrivée stade impasse banque souverain etat exsangue vider caisse payant innombrable croupier emploi garantir tout partie injectant argent client système piper distribuer subside joueur", 'image image j lettre ci joindre image', 'rattention Monsieur maire mardi janvier Monsieur maire demande évêque monseig

In [5]:
# possible batch
rows, columns = df.shape
print(rows//1000)


225
